In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg

import os.path as op
import time


from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from keras_tqdm import TQDMNotebookCallback
import numpy as np

from cascading import cascade_net
from data import MaskedUntouched2DSequence
from evaluate import psnr

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# paths
train_path = '/media/Zaccharie/UHRes/singlecoil_train/singlecoil_train/'
val_path = '/media/Zaccharie/UHRes/singlecoil_val/'
test_path = '/media/Zaccharie/UHRes/singlecoil_test/'

n_samples_train = 7135
n_samples_val = 7135

n_volumes_train = 199
n_volumes_val = 199

In [4]:
# generators
AF = 4
# MaskShifted2DSequence, MaskShiftedSingleImage2DSequence, MaskedUntouched2DSequence
train_gen = MaskedUntouched2DSequence(train_path, af=AF, inner_slices=1)
val_gen = MaskedUntouched2DSequence(val_path, af=AF)

In [5]:
run_params = {
    'n_cascade': 5, 
    'n_convs': 5, 
    'n_filters': 64,
    'noiseless': True,
}

n_epochs = 10
run_id = f'cascadenet_af{AF}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
print(run_id)

cascadenet_af4_1566912881


In [6]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=2, save_weights_only=True)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
)
lr_on_plat_cback = ReduceLROnPlateau(monitor='val_keras_psnr', min_lr=5*1e-5, mode='auto', patience=3)
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")

In [7]:
model = cascade_net(lr=1e-3, **run_params)
print(model.summary(line_length=100))

W0827 15:34:42.249148 139812472858368 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:529: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0827 15:34:42.258141 139812472858368 deprecation.py:323] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/pdnet_crop.py:63: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0827 15:34:42.285768 139812472858368 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0827 15:34:42.293187 139812472858368 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:4467: The name tf.truncated_normal is deprecated. Please use tf.ra

Model: "model_1"
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
kspace_input (InputLayer)        (None, 640, None, 1)  0                                            
____________________________________________________________________________________________________
ifft_simple (Lambda)             (None, 640, None, 1)  0           kspace_input[0][0]               
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 640, None, 1)  0           ifft_simple[0][0]                
____________________________________________________________________________________________________
lambda_2 (Lambda)                (None, 640, None, 1)  0           ifft_simple[0][0]                
__________________________________________________________________________

In [8]:
%%time
# simple overfit trials
data = train_gen[0]
val_data = val_gen[1]
model.fit(
    x=data[0], 
    y=data[1], 
#     validation_data=val_data, 
    batch_size=data[0][0].shape[0], 
    epochs=500,
    verbose=2, 
    shuffle=False,
)

W0827 15:34:44.395013 139812472858368 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:1021: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0827 15:34:44.734767 139812472858368 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:1008: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/500
 - 5s - loss: 1.5018e-05 - mean_squared_error: 4.3692e-10 - keras_psnr: 32.0285 - keras_ssim: 0.6856
Epoch 2/500
 - 0s - loss: 1.2972e-05 - mean_squared_error: 3.1809e-10 - keras_psnr: 33.4071 - keras_ssim: 0.7354
Epoch 3/500
 - 0s - loss: 1.1431e-05 - mean_squared_error: 2.4271e-10 - keras_psnr: 34.5816 - keras_ssim: 0.7777
Epoch 4/500
 - 0s - loss: 1.0409e-05 - mean_squared_error: 1.9873e-10 - keras_psnr: 35.4499 - keras_ssim: 0.8078
Epoch 5/500
 - 0s - loss: 9.7075e-06 - mean_squared_error: 1.7154e-10 - keras_psnr: 36.0889 - keras_ssim: 0.8289
Epoch 6/500
 - 0s - loss: 9.2010e-06 - mean_squared_error: 1.5337e-10 - keras_psnr: 36.5752 - keras_ssim: 0.8442
Epoch 7/500
 - 0s - loss: 8.8116e-06 - mean_squared_error: 1.4043e-10 - keras_psnr: 36.9579 - keras_ssim: 0.8557
Epoch 8/500
 - 0s - loss: 8.5056e-06 - mean_squared_error: 1.3085e-10 - keras_psnr: 37.2647 - keras_ssim: 0.8645
Epoch 9/500
 - 0s - loss: 8.2440e-06 - mean_squared_error: 1.2310e-10 - keras_psnr: 37.5299 - ke

Epoch 73/500
 - 0s - loss: 6.0971e-06 - mean_squared_error: 6.4112e-11 - keras_psnr: 40.3631 - keras_ssim: 0.9227
Epoch 74/500
 - 0s - loss: 6.0892e-06 - mean_squared_error: 6.3927e-11 - keras_psnr: 40.3757 - keras_ssim: 0.9229
Epoch 75/500
 - 0s - loss: 6.0814e-06 - mean_squared_error: 6.3737e-11 - keras_psnr: 40.3886 - keras_ssim: 0.9230
Epoch 76/500
 - 0s - loss: 6.0737e-06 - mean_squared_error: 6.3542e-11 - keras_psnr: 40.4020 - keras_ssim: 0.9231
Epoch 77/500
 - 0s - loss: 6.0661e-06 - mean_squared_error: 6.3347e-11 - keras_psnr: 40.4153 - keras_ssim: 0.9233
Epoch 78/500
 - 0s - loss: 6.0588e-06 - mean_squared_error: 6.3157e-11 - keras_psnr: 40.4284 - keras_ssim: 0.9234
Epoch 79/500
 - 0s - loss: 6.0516e-06 - mean_squared_error: 6.2974e-11 - keras_psnr: 40.4409 - keras_ssim: 0.9236
Epoch 80/500
 - 0s - loss: 6.0446e-06 - mean_squared_error: 6.2800e-11 - keras_psnr: 40.4530 - keras_ssim: 0.9237
Epoch 81/500
 - 0s - loss: 6.0376e-06 - mean_squared_error: 6.2633e-11 - keras_psnr: 40.

Epoch 145/500
 - 0s - loss: 5.7377e-06 - mean_squared_error: 5.6145e-11 - keras_psnr: 40.9394 - keras_ssim: 0.9289
Epoch 146/500
 - 0s - loss: 5.7342e-06 - mean_squared_error: 5.6079e-11 - keras_psnr: 40.9446 - keras_ssim: 0.9290
Epoch 147/500
 - 0s - loss: 5.7308e-06 - mean_squared_error: 5.6010e-11 - keras_psnr: 40.9499 - keras_ssim: 0.9290
Epoch 148/500
 - 0s - loss: 5.7273e-06 - mean_squared_error: 5.5944e-11 - keras_psnr: 40.9550 - keras_ssim: 0.9291
Epoch 149/500
 - 0s - loss: 5.7238e-06 - mean_squared_error: 5.5879e-11 - keras_psnr: 40.9601 - keras_ssim: 0.9292
Epoch 150/500
 - 0s - loss: 5.7204e-06 - mean_squared_error: 5.5814e-11 - keras_psnr: 40.9651 - keras_ssim: 0.9292
Epoch 151/500
 - 0s - loss: 5.7170e-06 - mean_squared_error: 5.5749e-11 - keras_psnr: 40.9702 - keras_ssim: 0.9293
Epoch 152/500
 - 0s - loss: 5.7136e-06 - mean_squared_error: 5.5684e-11 - keras_psnr: 40.9753 - keras_ssim: 0.9293
Epoch 153/500
 - 0s - loss: 5.7102e-06 - mean_squared_error: 5.5619e-11 - keras_

Epoch 217/500
 - 0s - loss: 5.5207e-06 - mean_squared_error: 5.2207e-11 - keras_psnr: 41.2552 - keras_ssim: 0.9326
Epoch 218/500
 - 0s - loss: 5.5181e-06 - mean_squared_error: 5.2161e-11 - keras_psnr: 41.2591 - keras_ssim: 0.9326
Epoch 219/500
 - 0s - loss: 5.5154e-06 - mean_squared_error: 5.2117e-11 - keras_psnr: 41.2627 - keras_ssim: 0.9326
Epoch 220/500
 - 0s - loss: 5.5127e-06 - mean_squared_error: 5.2072e-11 - keras_psnr: 41.2665 - keras_ssim: 0.9327
Epoch 221/500
 - 0s - loss: 5.5101e-06 - mean_squared_error: 5.2031e-11 - keras_psnr: 41.2699 - keras_ssim: 0.9327
Epoch 222/500
 - 0s - loss: 5.5076e-06 - mean_squared_error: 5.1984e-11 - keras_psnr: 41.2739 - keras_ssim: 0.9328
Epoch 223/500
 - 0s - loss: 5.5050e-06 - mean_squared_error: 5.1942e-11 - keras_psnr: 41.2773 - keras_ssim: 0.9328
Epoch 224/500
 - 0s - loss: 5.5024e-06 - mean_squared_error: 5.1899e-11 - keras_psnr: 41.2810 - keras_ssim: 0.9329
Epoch 225/500
 - 0s - loss: 5.4998e-06 - mean_squared_error: 5.1857e-11 - keras_

Epoch 289/500
 - 0s - loss: 5.3456e-06 - mean_squared_error: 4.9332e-11 - keras_psnr: 41.5013 - keras_ssim: 0.9355
Epoch 290/500
 - 0s - loss: 5.3438e-06 - mean_squared_error: 4.9303e-11 - keras_psnr: 41.5038 - keras_ssim: 0.9355
Epoch 291/500
 - 0s - loss: 5.3417e-06 - mean_squared_error: 4.9270e-11 - keras_psnr: 41.5067 - keras_ssim: 0.9355
Epoch 292/500
 - 0s - loss: 5.3397e-06 - mean_squared_error: 4.9230e-11 - keras_psnr: 41.5103 - keras_ssim: 0.9356
Epoch 293/500
 - 0s - loss: 5.3369e-06 - mean_squared_error: 4.9197e-11 - keras_psnr: 41.5132 - keras_ssim: 0.9356
Epoch 294/500
 - 0s - loss: 5.3342e-06 - mean_squared_error: 4.9155e-11 - keras_psnr: 41.5168 - keras_ssim: 0.9356
Epoch 295/500
 - 0s - loss: 5.3321e-06 - mean_squared_error: 4.9125e-11 - keras_psnr: 41.5195 - keras_ssim: 0.9357
Epoch 296/500
 - 0s - loss: 5.3300e-06 - mean_squared_error: 4.9093e-11 - keras_psnr: 41.5224 - keras_ssim: 0.9357
Epoch 297/500
 - 0s - loss: 5.3283e-06 - mean_squared_error: 4.9052e-11 - keras_

Epoch 361/500
 - 0s - loss: 5.1902e-06 - mean_squared_error: 4.7000e-11 - keras_psnr: 41.7116 - keras_ssim: 0.9380
Epoch 362/500
 - 0s - loss: 5.1884e-06 - mean_squared_error: 4.6977e-11 - keras_psnr: 41.7137 - keras_ssim: 0.9380
Epoch 363/500
 - 0s - loss: 5.1867e-06 - mean_squared_error: 4.6944e-11 - keras_psnr: 41.7167 - keras_ssim: 0.9381
Epoch 364/500
 - 0s - loss: 5.1853e-06 - mean_squared_error: 4.6926e-11 - keras_psnr: 41.7184 - keras_ssim: 0.9381
Epoch 365/500
 - 0s - loss: 5.1841e-06 - mean_squared_error: 4.6906e-11 - keras_psnr: 41.7203 - keras_ssim: 0.9381
Epoch 366/500
 - 0s - loss: 5.1832e-06 - mean_squared_error: 4.6869e-11 - keras_psnr: 41.7237 - keras_ssim: 0.9382
Epoch 367/500
 - 0s - loss: 5.1800e-06 - mean_squared_error: 4.6844e-11 - keras_psnr: 41.7260 - keras_ssim: 0.9382
Epoch 368/500
 - 0s - loss: 5.1766e-06 - mean_squared_error: 4.6796e-11 - keras_psnr: 41.7304 - keras_ssim: 0.9382
Epoch 369/500
 - 0s - loss: 5.1738e-06 - mean_squared_error: 4.6760e-11 - keras_

Epoch 433/500
 - 0s - loss: 5.0511e-06 - mean_squared_error: 4.5039e-11 - keras_psnr: 41.8967 - keras_ssim: 0.9403
Epoch 434/500
 - 0s - loss: 5.0516e-06 - mean_squared_error: 4.5035e-11 - keras_psnr: 41.8970 - keras_ssim: 0.9403
Epoch 435/500
 - 0s - loss: 5.0469e-06 - mean_squared_error: 4.4984e-11 - keras_psnr: 41.9020 - keras_ssim: 0.9404
Epoch 436/500
 - 0s - loss: 5.0411e-06 - mean_squared_error: 4.4921e-11 - keras_psnr: 41.9080 - keras_ssim: 0.9405
Epoch 437/500
 - 0s - loss: 5.0387e-06 - mean_squared_error: 4.4904e-11 - keras_psnr: 41.9097 - keras_ssim: 0.9405
Epoch 438/500
 - 0s - loss: 5.0359e-06 - mean_squared_error: 4.4848e-11 - keras_psnr: 41.9151 - keras_ssim: 0.9405
Epoch 439/500
 - 0s - loss: 5.0331e-06 - mean_squared_error: 4.4835e-11 - keras_psnr: 41.9164 - keras_ssim: 0.9406
Epoch 440/500
 - 0s - loss: 5.0315e-06 - mean_squared_error: 4.4802e-11 - keras_psnr: 41.9196 - keras_ssim: 0.9406
Epoch 441/500
 - 0s - loss: 5.0307e-06 - mean_squared_error: 4.4786e-11 - keras_